**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Jun 25 14:49:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Annuaire Simplon.gsheet'
'Colab Notebooks'
'Evaluations individuelles.xlsx'
 ex.py
'fait COACHING Promo simplon 2020.gsheet'
'File Rouge'
'Lois de probabilités.pdf'
'Supports Noura'
'test statistique.gslides'
 train.csv
 Tuto.txt
'Types de Graphiques Statistiques.gslides'
'Untitled document.gdoc'
 yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 13763 (delta 11), reused 5 (delta 1), pack-reused 13738
Receiving objects: 100% (13763/13763), 12.34 MiB | 18.50 MiB/s, done.
Resolving deltas: 100% (9383/9383), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-25 14:52:44--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   300KB/s    in 7m 41s  

2020-06-25 15:00:27 (344 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/all.zip -d data/obj

Archive:  /mydrive/yolov3/all.zip
  inflating: data/obj/85.png         
  inflating: data/obj/85.txt         
  inflating: data/obj/86.png         
  inflating: data/obj/86.txt         
  inflating: data/obj/87.png         
  inflating: data/obj/87.txt         
  inflating: data/obj/88.png         
  inflating: data/obj/88.txt         
  inflating: data/obj/89.png         
  inflating: data/obj/89.txt         
  inflating: data/obj/90.png         
  inflating: data/obj/90.txt         
  inflating: data/obj/91.png         
  inflating: data/obj/91.txt         
  inflating: data/obj/25.png         
  inflating: data/obj/25.txt         
  inflating: data/obj/27.png         
  inflating: data/obj/27.txt         
  inflating: data/obj/28.png         
  inflating: data/obj/28.txt         
  inflating: data/obj/29.png         
  inflating: data/obj/29.txt         
  inflating: data/obj/30.png         
  inflating: data/obj/30.txt         
  inflating: data/obj/31.png         
  inflating: dat

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

['15', '0.319363', '0.306641', '0.061079', '0.222656']
24 data/obj/79.txt
0 0.319363 0.306641 0.061079 0.222656
15 0.359812 0.289844 0.039961 0.225000

['15', '0.359812', '0.289844', '0.039961', '0.225000']
24 data/obj/79.txt
0 0.359812 0.289844 0.039961 0.225000
15 0.391975 0.282031 0.041910 0.253125

['15', '0.391975', '0.282031', '0.041910', '0.253125']
24 data/obj/79.txt
0 0.391975 0.282031 0.041910 0.253125
15 0.431611 0.261328 0.040611 0.266406

['15', '0.431611', '0.261328', '0.040611', '0.266406']
24 data/obj/79.txt
0 0.431611 0.261328 0.040611 0.266406
15 0.472872 0.252344 0.054906 0.287500

['15', '0.472872', '0.252344', '0.054906', '0.287500']
24 data/obj/79.txt
0 0.472872 0.252344 0.054906 0.287500
15 0.520955 0.260937 0.049058 0.250000

['15', '0.520955', '0.260937', '0.049058', '0.250000']
24 data/obj/79.txt
0 0.520955 0.260937 0.049058 0.250000
15 0.561241 0.266797 0.042560 0.219531

['15', '0

In [12]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/48.png', 'data/obj/28.png', 'data/obj/71.png', 'data/obj/77.png', 'data/obj/83.png', 'data/obj/29.png', 'data/obj/72.png', 'data/obj/25.png', 'data/obj/44.png', 'data/obj/56.png', 'data/obj/68.png', 'data/obj/40.png', 'data/obj/63.png', 'data/obj/33.png', 'data/obj/88.png', 'data/obj/91.png', 'data/obj/67.png', 'data/obj/70.png', 'data/obj/46.png', 'data/obj/78.png', 'data/obj/31.png', 'data/obj/42.png', 'data/obj/76.png', 'data/obj/39.png', 'data/obj/57.png', 'data/obj/79.png', 'data/obj/84.png', 'data/obj/66.png', 'data/obj/82.png', 'data/obj/62.png', 'data/obj/30.png', 'data/obj/45.png', 'data/obj/73.png', 'data/obj/80.png', 'data/obj/85.png', 'data/obj/86.png', 'data/obj/47.png', 'data/obj/69.png', 'data/obj/37.png', 'data/obj/60.png', 'data/obj/89.png', 'data/obj/32.png', 'data/obj/90.png', 'data/obj/81.png', 'data/obj/43.png', 'data/obj/38.png', 'data/obj/36.png', 'data/obj/75.png', 'data/obj/65.png', 'data/obj/61.png', 'data/obj/35.png', 'data/obj/27.png', 'data/obj/8

In [13]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

 81: 1150.670410, 886.320984 avg loss, 0.000000 rate, 20.242037 seconds, 5184 images, 22.166740 hours left
Loaded: 0.000053 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.526415, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 253.735504, iou_loss = 0.000000, total_loss = 253.735504 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.290249, GIOU: 0.210062), Class: 0.543549, Obj: 0.397666, No Obj: 0.394367, .5R: 0.119658, .75R: 0.008547, count: 117, class_loss = 596.388184, iou_loss = 111.323975, total_loss = 707.712158 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.401487, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 2438.819580, iou_loss = 0.000000, total_loss = 2438.819580 
 total_b